# Keras에서 모델 저장하고 불러오기 - Image classification with MNIST dataset
MNIST image classification 코드 예제를 통해 tf.keras.callbacks.ModelCheckpoint를 이용하여 모델을 저장하고 불러오는 방법을 설명합니다.

### 라이브러리 import (h5py, tensorflow, keras 설치 필요)

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

import os
import h5py
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

1.14.0


/home/ubuntu/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorboard/compat/ten

### MNIST 데이터셋 다운로드
데이터셋 이미지 중 첫 1000개만 사용

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### 모델 만들기

In [0]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model


model = create_model()
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### 체크포인트 콜백 사용하여 체크포인트 저장, 학습 시작
체크포인트 파일을 만들고 매 epoch 종료시 파일 업데이트


In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model = create_model()

model. fit(train_images, train_labels, epochs = 10,
           validation_data = (test_images, test_labels),
           callbacks = [cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 544/1000 [===============>..............] - ETA: 0s - loss: 1.4692 - acc: 0.5809
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 348us/sample - loss: 1.1189 - acc: 0.6800 - val_loss: 0.7396 - val_acc: 0.7700
Epoch 2/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.4708 - acc: 0.8785
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 153us/sample - loss: 0.4443 - acc: 0.8810 - val_loss: 0.5552 - val_acc: 0.8120
Epoch 3/10
 544/1000 [===============>..............] - ETA: 0s - loss: 0.2701 - acc: 0.9357
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 154us/sample - loss: 0.2868 - acc: 0.9270 - val_loss: 0.4688 - val_acc: 0.8500
Epoch 4/10
 544/1000 [===============>..............] - ETA: 0s - loss: 0.1821 - acc: 0.9651
Epoch 00004: saving model to training_1/cp.ckpt
1000/10

### 훈련되지 않은 새로운 모델 생성하여 테스트셋으로 성능 평가

In [0]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("학습되지 않은 모델 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 67us/sample - loss: 2.3299 - acc: 0.1270
훈련되지 않은 모델의 정확도: 12.70%


### 저장된 체크포인트를 로드하여 다시 평가

In [0]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("복원된 모델 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 37us/sample - loss: 0.4161 - acc: 0.8680
복원된 모델의 정확도: 86.80%
